This assignment is inspired by: 

- https://www.kaggle.com/code/carlmcbrideellis/an-introduction-to-xgboost-regression
- https://www.kaggle.com/code/dansbecker/xgboost/notebook

In this assignment we will apply XGBoost Regression techniques to predict house prices, based on the famous Kaggle Dataset https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques

Step 1 is to download the dataset.

In [45]:
#=========================================================================
# load up the libraries
#=========================================================================
import pandas  as pd
import numpy   as np
import xgboost as xgb

#=========================================================================
# read in the data
#=========================================================================
train_data = pd.read_csv('train.csv',index_col=0)
test_data  = pd.read_csv('test.csv',index_col=0)

### <center style="background-color:Gainsboro; width:60%;">Feature selection</center>
The purpose of feature selection, as the name suggests, is to only model the most pertinent and important features, thus reducing the computational overhead, and also to alleviate the [curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality). The following are a number of notebooks covering techniques to achieve said goal, all of which use the House Prices data as an example:

* [Feature selection using the Boruta-SHAP package](https://www.kaggle.com/carlmcbrideellis/feature-selection-using-the-boruta-shap-package)
* [Recursive Feature Elimination (RFE) example](https://www.kaggle.com/carlmcbrideellis/recursive-feature-elimination-rfe-example)
* [House Prices: Permutation Importance example](https://www.kaggle.com/carlmcbrideellis/house-prices-permutation-importance-example)
* [Feature importance using the LASSO](https://www.kaggle.com/carlmcbrideellis/feature-importance-using-the-lasso)

In this assignment, we shall use all of the numerical columns, and ignore the categorical features. To encode the categorical features one can use for example [pandas.get_dummies](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html). 

Our first task is to do Feature Exploration and Selection. 

In [46]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [47]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 1461 to 2919
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1459 non-null   int64  
 1   MSZoning       1455 non-null   object 
 2   LotFrontage    1232 non-null   float64
 3   LotArea        1459 non-null   int64  
 4   Street         1459 non-null   object 
 5   Alley          107 non-null    object 
 6   LotShape       1459 non-null   object 
 7   LandContour    1459 non-null   object 
 8   Utilities      1457 non-null   object 
 9   LotConfig      1459 non-null   object 
 10  LandSlope      1459 non-null   object 
 11  Neighborhood   1459 non-null   object 
 12  Condition1     1459 non-null   object 
 13  Condition2     1459 non-null   object 
 14  BldgType       1459 non-null   object 
 15  HouseStyle     1459 non-null   object 
 16  OverallQual    1459 non-null   int64  
 17  OverallCond    1459 non-null   int64  
 18  YearB

In [48]:
#encode training data
categorical_cols = train_data.select_dtypes(exclude='number').columns
train_data = pd.get_dummies(train_data, columns=categorical_cols)

#encode testing data
categorical_cols = test_data.select_dtypes(exclude='number').columns
test_data = pd.get_dummies(test_data, columns=categorical_cols)


### <center style="background-color:Gainsboro; width:60%;">Feature engineering</center>
As mentioned, one aspect of feature engineering is the creation of new features out of existing features. A simple example would be to create a new feature which is the sum of the number of bathrooms in the house:

In [49]:
for df in (train_data, test_data):
    df["n_bathrooms"] = df["BsmtFullBath"] + (df["BsmtHalfBath"]*0.5) + df["FullBath"] + (df["HalfBath"]*0.5)
    df["area_with_basement"]  = df["GrLivArea"] + df["TotalBsmtSF"]

Your next task is to apply some feature engineering to prepare for using the XGBoost Estimator to predict house prices.

In [50]:
from sklearn.model_selection import train_test_split
X = train_data.drop(columns=['SalePrice'])
y = train_data['SalePrice']

X_train, X_test, y_train, y_test= train_test_split(X, y, random_state=42, shuffle=True)

For more on this fascinating aspect may I recommend the free on-line book ["*Feature Engineering and Selection: A Practical Approach for Predictive Models*"](http://www.feat.engineering/) by Max Kuhn and Kjell Johnson.
### <center style="background-color:Gainsboro; width:60%;">XGBoost estimator</center>
Note that for this competition we use the RMSLE evaluation metric, rather than the default metric, which for regression is the RMSE. For more on the peculiarities of the RMSLE see the Appendix below.

In [51]:
#=========================================================================
# XGBoost regression: 
# Parameters: 
# n_estimators  "Number of gradient boosted trees. Equivalent to number 
#                of boosting rounds."
# learning_rate "Boosting learning rate (also known as “eta”)"
# max_depth     "Maximum depth of a tree. Increasing this value will make 
#                the model more complex and more likely to overfit." 
#=========================================================================
from xgboost import XGBRegressor
regressor=xgb.XGBRegressor(eval_metric='rmsle')

#=========================================================================
# exhaustively search for the optimal hyperparameters
#=========================================================================
from sklearn.model_selection import GridSearchCV, RepeatedKFold, RandomizedSearchCV
# set up our search grid
param_grid = {"max_depth":    [4, 5],
              "n_estimators": [500, 600, 700],
              "learning_rate": [0.01, 0.015]}

param_dist = {
    'n_estimators': [50, 100, 150, 200, 250],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'min_child_weight': [1, 3, 5, 7],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'gamma': [0.0, 0.1, 0.2, 0.3],
    'reg_alpha': [0.0, 0.1, 0.2, 0.3],
    'reg_lambda': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]}

In [52]:
from sklearn.feature_selection import RFECV
# here we want only one final feature, we do this to produce a ranking
n_features_to_select = 1
rfecv = RFECV(estimator=regressor, scoring='neg_mean_squared_error', cv=5)
rfecv.fit(X_train, y_train)

# Get the selected features from RFECV
selected_features = X_train.columns[rfecv.support_].sort_values()

X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Initialize and fit the XGBRegressor model using the subsetted training data
regressor_selected_features = XGBRegressor(eval_metric='rmsle')

In [53]:
from sklearn.metrics import get_scorer_names

# Get the list of available scoring metric names
scorer_names = get_scorer_names()

# Print the list of scorer names
print(scorer_names)

['accuracy', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'average_precision', 'balanced_accuracy', 'completeness_score', 'explained_variance', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'fowlkes_mallows_score', 'homogeneity_score', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted', 'matthews_corrcoef', 'max_error', 'mutual_info_score', 'neg_brier_score', 'neg_log_loss', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_gamma_deviance', 'neg_mean_poisson_deviance', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error', 'neg_negative_likelihood_ratio', 'neg_root_mean_squared_error', 'normalized_mutual_info_score', 'positive_likelihood_ratio', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'rand_score', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'roc_auc', 'roc_auc_ovo', 'roc_auc_ovo_weight

Can you use grid search to find the optimal hyper parameters?

In [54]:
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)

grid_search = GridSearchCV(estimator=regressor, scoring='neg_mean_squared_error', param_grid=param_grid ,cv=cv)

grid_search.fit(X_train_selected, y_train)

GridSearchCV(cv=RepeatedKFold(n_repeats=3, n_splits=5, random_state=42),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False,
                                    eval_metric='rmsle', feature_types=None,
                                    gamma=None, gpu_id=None, grow_policy=None,
                                    importance_type...
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             param_grid={'learning_rate': [0.01, 0.015], 'max_depth': [4, 5],
                         'n_estimators': [500, 600, 700]},
             scoring='neg_mean_squared_error')

In [55]:
random_search = RandomizedSearchCV(estimator=regressor, param_distributions=param_dist, scoring='neg_mean_squared_error', n_iter=25, cv=cv, random_state=42 )

random_search.fit(X_train_selected, y_train)

RandomizedSearchCV(cv=RepeatedKFold(n_repeats=3, n_splits=5, random_state=42),
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric='rmsle',
                                          feature_types=None, gamma=None,
                                          gpu_id=None, grow_policy=None,
                                          importanc...
                   n_iter=25,
                   param_distributions={'colsample_bytree': [0.8, 0.9, 1.0],
                                        'gamma': [0.0, 0.1, 0.2, 0.3],
                                        'learning_rate': [0.01, 0.1, 0.2, 0.3],
                                        'max_depth': [3, 5, 7, 9],
                                        'min_child_weight': [1, 3, 5, 7],
                                        'n_estimators': [50, 100, 150, 200,
                                                         250],
                                        'reg_alpha': [0.0, 0.1, 0.2, 0.3],
                                        'reg_lambda': [0.0, 0.1, 0.2, 0.3, 0.4,
                                                       0.5],
                                        'subsample': [0.8, 0.9, 1.0]},
                   random_state=42, scoring='neg_mean_squared_error')

In [56]:
print(f"The best hyperparameters are: {grid_search.best_params_}")
print(f"The best hyperparameters are: {random_search.best_params_}")

The best hyperparameters are: {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 700}
The best hyperparameters are: {'subsample': 1.0, 'reg_lambda': 0.5, 'reg_alpha': 0.1, 'n_estimators': 50, 'min_child_weight': 7, 'max_depth': 9, 'learning_rate': 0.1, 'gamma': 0.0, 'colsample_bytree': 0.9}


Now, can you setup a XGBoost Regressor object using your hyperparameters and fit it?

In [61]:
regressor_grid = XGBRegressor(learning_rate=0.015, max_depth=5, n_estimators=700, eval_metric='rmsle')
regressor_random = XGBRegressor(subsample=1.0, reg_lambda=0.5, reg_alpha=0.1, n_estimators=50, 
                                min_child_weight=7, max_depth=9, learning_rate=0.1, gamma=0.0, colsample_bytree=0.9, eval_metric='rmsle')

regressor_grid.fit(X_train_selected, y_train)
regressor_random.fit(X_train_selected, y_train)

y_pred_grid = regressor_grid.predict(X_test_selected)
y_pred_rand = regressor_random.fit(X_train_selected, y_train)



Finally, can you run it on your test set?

In [64]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, explained_variance_score, mean_squared_log_error
# Calculate the metrics for y_pred_grid
mse_grid = mean_squared_error(y_test, y_pred_grid)
r2_grid = r2_score(y_test, y_pred_grid)
mae_grid = mean_absolute_error(y_test, y_pred_grid)
evs_grid = explained_variance_score(y_test, y_pred_grid)
msle_grid = mean_squared_log_error(y_test, y_pred_grid)

# Calculate the metrics for y_pred_rand
y_pred_rand = regressor_random.predict(X_test_selected)  # If not already predicted
mse_rand = mean_squared_error(y_test, y_pred_rand)
r2_rand = r2_score(y_test, y_pred_rand)
mae_rand = mean_absolute_error(y_test, y_pred_rand)
evs_rand = explained_variance_score(y_test, y_pred_rand)
msle_rand = mean_squared_log_error(y_test, y_pred_rand)

# Print the results
print("Results for y_pred_grid:")
print("Mean Squared Error:", mse_grid)
print("R-squared (R2):", r2_grid)
print("Mean Absolute Error:", mae_grid)
print("Explained Variance Score:", evs_grid)
print('Mean Squared Log Error:', msle_grid)

print("\nResults for y_pred_rand:")
print("Mean Squared Error:", mse_rand)
print("R-squared (R2):", r2_rand)
print("Mean Absolute Error:", mae_rand)
print("Explained Variance Score:", evs_rand)
print('Mean Squared Log Error:', msle_rand)


Results for y_pred_grid:
Mean Squared Error: 1098472149.6111753
R-squared (R2): 0.8431943342678998
Mean Absolute Error: 18368.889437071917
Explained Variance Score: 0.8432285629110926
Mean Squared Log Error: 0.024147549305528534

Results for y_pred_rand:
Mean Squared Error: 874593337.7516903
R-squared (R2): 0.8751527832366464
Mean Absolute Error: 18191.092626284248
Explained Variance Score: 0.8752256870024118
Mean Squared Log Error: 0.022539487381099493


In [59]:
#Can you score your solution offline and see how it does?

In [63]:
# read in the ground truth file
solution   = pd.read_csv(<your solution file>)
y_true     = solution["SalePrice"]

from sklearn.metrics import mean_squared_log_error
RMSLE = np.sqrt( mean_squared_log_error(y_true, solution))
print("The score is %.5f" % RMSLE )

SyntaxError: invalid syntax (2390037664.py, line 2)

Finally, use the below block to prepare your submission

In [ ]:
output = pd.DataFrame({"Id":test_data.index, "SalePrice":solution})
output.to_csv('submission.csv', index=False)

### <center style="background-color:Gainsboro; width:60%;">Feature importance</center>
Let us also take a very quick look at the feature importance too:

In [ ]:
from xgboost import plot_importance
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams.update({'font.size': 16})

fig, ax = plt.subplots(figsize=(12,6))
plot_importance(regressor, max_num_features=8, ax=ax)
plt.show();

Where here the `F score` is a measure "*...based on the number of times a variable is selected for splitting, weighted by the squared improvement to the model as a result of each split, and averaged over all trees*." [1] 

Note that these importances are susceptible to small changes in the training data, and it is much better to make use of ["GPU accelerated SHAP values"](https://www.kaggle.com/carlmcbrideellis/gpu-accelerated-shap-values-jane-street-example), incorporated with version 1.3 of XGBoost.

Can you follow the above guide use SHAP values instead of F Score?

In [ ]:
# code here

### <center style="background-color:Gainsboro; width:60%;">Appendix: The RMSLE evaluation metric</center>
From the competition [evaluation page](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview/evaluation) we see that the metric we are using is the root mean squared logarithmic error (RMSLE), which is given by

$$ {\mathrm {RMSLE}}\,(y, \hat y) = \sqrt{ \frac{1}{n} \sum_{i=1}^n \left(\log (1 + \hat{y}_i) - \log (1 + y_i)\right)^2} $$

where $\hat{y}_i$ is the predicted value of the target for instance $i$, and $y_i$
is the actual value of the target for instance $i$.

It is important to note that, unlike the RMSE, the RMSLE is asymmetric; penalizing much more the underestimated predictions than the overestimated predictions. For example, say the correct value is $y_i = 1000$, then underestimating by 600 is almost twice as bad as overestimating by 600:

In [ ]:
def RSLE(y_hat,y):
    return np.sqrt((np.log1p(y_hat) - np.log1p(y))**2)

print("The RMSLE score is %.3f" % RSLE( 400,1000) )
print("The RMSLE score is %.3f" % RSLE(1600,1000) )

The asymmetry arises because 

$$ \log (1 + \hat{y}_i) - \log (1 + y_i) =  \log \left( \frac{1 + \hat{y}_i}{1 + y_i} \right) $$

so we are essentially looking at ratios, rather than differences such as is the case of the RMSE. We can see the form that this asymmetry takes in the following plot, again using 1000 as our ground truth value:

In [ ]:
plt.rcParams["figure.figsize"] = (7, 4)
x = np.linspace(5,4000,100)
plt.plot(x, RSLE(x,1000))
plt.xlabel('prediction')
plt.ylabel('RMSLE')
plt.show()